# Validation of the PyPSA-Africa generation by hydro powerplants

This notebooks investigates the total energy generation by hydro powerplants by the workflow

To reproduce the findings obtained in this notebook,
please run the full snakemake workflow for the Africa.
To do so, please set ``countries = ["Africa"]`` in the ``config.yaml`` file.

Note. The solved network is required as the total generation dispatch of the optimized network is considered. 